<a href="https://colab.research.google.com/github/kamalzada/Machine-Learning-Projects/blob/master/fighter_SLpM_scraped.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
# For web scraping
import requests
from bs4 import BeautifulSoup

# For data cleaning/feature engineering
import numpy as np
import pandas as pd

# For visualization
import seaborn as sns
import matplotlib.pyplot as plt

#For statistical tests
import pingouin as pg

#Miscellaneous
import string

In [ ]:
letters = string.ascii_lowercase
fighter_stats = {}

for letter in letters:
    pages = requests.get(f'http://ufcstats.com/statistics/fighters?char={letter}&page=all')
    soup = BeautifulSoup(pages.text, 'lxml')   
    fighter_stats[letter] = (soup.find_all('td', attrs = {'class': "b-statistics__table-col"}))  

In [80]:
fighter_stats['b'][0].text.split('\n')[1]

'Niklas'

In [ ]:
fighter_first_names = []

for key, value in fighter_stats.items():
    for fighter in np.arange(0, len(fighter_stats[key]), 11):
        fighter_first_name = value[fighter].text.split("\n")[1]
        fighter_first_names.append(fighter_first_name)
fighter_first_names = pd.Series(fighter_first_names)        

In [ ]:
fighter_last_names = []

for key, value in fighter_stats.items():
    for fighter in np.arange(1, len(fighter_stats[key]) + 1, 11):
        fighter_last_name = value[fighter].text.split("\n")[1]
        fighter_last_names.append(fighter_last_name)
fighter_last_names = pd.Series(fighter_last_names)        

In [ ]:
fighter_stances = []

for key, value in fighter_stats.items():
    for fighter in np.arange(6, len(fighter_stats[key]) + 6, 11):
        fighter_stance = value[fighter].text.split("\n")[1][10:]
        fighter_stances.append(fighter_stance)
fighter_stances = pd.Series(fighter_stances)        

In [ ]:
fighter_losses = []

for key, value in fighter_stats.items():
    for fighter in np.arange(8, len(fighter_stats[key]) + 8, 11):
        fighter_loss = value[fighter].text.split("\n")[1][10:]
        fighter_losses.append(fighter_loss)
fighter_losses = pd.Series(fighter_losses)

In [ ]:
fighter_wins = []

for key, value in fighter_stats.items():
    for fighter in np.arange(7, len(fighter_stats[key]) + 7, 11):
        fighter_win = value[fighter].text.split("\n")[1][10:]
        fighter_wins.append(fighter_win)
fighter_wins = pd.Series(fighter_wins)

In [ ]:
df = pd.concat([fighter_first_names, fighter_last_names, fighter_stances, fighter_wins, fighter_losses], 
          keys = ['name', 'last_name', 'stance', 'wins', 'losses'], axis=1)

In [ ]:
df.tail()

,name,last_name,stance,wins,losses
3736,Dave,Zitanick,,5,7
3737,Alex,Zuniga,,6,3
3738,George,Zuniga,,3,1
3739,Allan,Zuniga,Orthodox,13,1
3740,Virgil,Zwicker,,15,6


In [ ]:
#Missing stance records
df[df['stance']==''].shape[0]

808

In [ ]:
letters = string.ascii_lowercase
fighter_ids = {}

for letter in letters:
    pages = requests.get(f'http://ufcstats.com/statistics/fighters?char={letter}&page=all')
    soup = BeautifulSoup(pages.text, 'lxml')   
    fighter_ids[letter] = soup.find_all('a', attrs = {'class': "b-link b-link_style_black"}) 

In [ ]:
#Extract pure links to send request to
fighter_links = []

for letter in letters:
    for fighter in fighter_ids[letter]:
        fighter_links.append(fighter['href'])

In [ ]:
fighter_links = list(pd.Series(fighter_links).to_frame()[0].unique())

In [ ]:
#Number of fighter's links that we'll loop through
len(fighter_links)

3741

In [415]:
%%time
fighter_str = {}

for link in fighter_links: 
  pages = requests.get(f'{link}')
  soup = BeautifulSoup(pages.text, 'lxml')   
  fighter_str[link] = soup.find_all('li', attrs = {'class': "b-list__box-list-item b-list__box-list-item_type_block"}) 

CPU times: user 1min 44s, sys: 2.96 s, total: 1min 47s
Wall time: 20min 57s


In [416]:
#Figuring out which row to choose just like previous
fighter_str['http://ufcstats.com/fighter-details/1c5879330d42255f'][5].text.split('\n')[5][9:] 

' 3.52'

In [417]:
SLpM = []

for link in fighter_links:
  SLpM.append(fighter_str[link][5].text.split('\n')[5][9:])  

In [420]:
SLpM = [f.strip() for f in SLpM]

In [422]:
df['SLpM'] = pd.Series(SLpM).to_frame()  

In [424]:
#Checking the values with the website. No problem.
df[df['name']=='Israel']

,name,last_name,stance,wins,losses,SLpM
14,Israel,Adesanya,Switch,22,1,3.84
40,Israel,Albuquerque,Orthodox,0,3,0.32
769,Israel,Delgado,,0,2,0.00
1161,Israel,Giron,,19,5,0.00


In [428]:
df['SLpM'] = df['SLpM'].astype('float')
df['wins'] = df['wins'].astype('int') 
df['losses'] = df['losses'].astype('int')  

In [430]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3741 entries, 0 to 3740
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   name       3741 non-null   object 
 1   last_name  3741 non-null   object 
 2   stance     3741 non-null   object 
 3   wins       3741 non-null   int64  
 4   losses     3741 non-null   int64  
 5   SLpM       3741 non-null   float64
dtypes: float64(1), int64(2), object(3)
memory usage: 175.5+ KB


In [450]:
df['SLpM'].isna().sum() 

0

In [449]:
'''Since we consider stats fighters with fewer than 10 fights somewhat unreliable,
let's check what percentage of records that we're gonna discard due to no stance info are fighters
with less than 10 fights'''

df['fights'] = df['wins'] + df['losses'] 
over = [f for f in df[df['stance']=='']['fights'].values if f<10] 
len(over)/len(list(df[df['stance']=='']['fights'].values)) 
#So almost a half

0.4715346534653465

In [435]:
#We'll start some statistical tests from here on on Jupyter notebook
df.groupby('stance')['SLpM'].mean()  

stance
               1.078057
Open Stance    1.717143
Orthodox       2.684936
Sideways       0.093333
Southpaw       2.535807
Switch         3.300479
Name: SLpM, dtype: float64

In [448]:
from google.colab import files
df.to_csv('fighter_stance.csv', index=False) 
files.download('fighter_stance.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>